In [183]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [184]:
import tensorflow as tf
from keras.layers import Dense , Activation , Dropout, Embedding, SpatialDropout1D, LSTM , TextVectorization, TimeDistributed
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

In [185]:
from bs4 import BeautifulSoup
import os
import re

In [186]:
directory = 'corpus_morphological_analysis'
file_paths = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    file_paths.append(f)    

In [187]:
temp_file_paths = file_paths[:100]

In [188]:
content = []
#i = 0
for filepath in temp_file_paths :
    #print(i)
    with open(filepath, encoding='utf-8') as f :
        html = f.read()
    soup = BeautifulSoup(html, features="html.parser")
    for script in soup(["script", "style"]):
        script.extract()  
    text = soup.get_text()
    content.append(text)
    #i+=1

In [260]:
content[0]

'مسقط:مُسْقَطٍ:#:مسقط:اسم زمان أو مكان:مُفْعَلٍ:سقط:#:\nالعمانية: لا توجد نتائج لتحليل هذه الكلمة\nيقام:يُقَامَ:ي: حرف المضارعة:يقام:فعل مضارع مبني للمجهول:يُفاعَ:قمي:#:\nفى: لا توجد نتائج لتحليل هذه الكلمة\nالرابع:الرَّابِعِ:ال: التعريف:رابع:اسم فاعل:فَاعِلِ:ربع:#:\nوالعشرين:وَالْعُشْرَيْنِ:وَال: حرف العطف+ال التعريف:عشرين:اسم جامد:فُعْلَيْنِ:عشر:ين:علامة الإعراب:\nمن:مَنْ:# :من:اسم شَرْط:#:#:# :\nشهر:شَهِّرْ:#:شهر:فعل أمر:فَعِّلْ:شهر:#:\nفبراير:فَبِرَايِرٍ:فَبِ: حرف العطف+حرف الجر:راير:اسم فاعل:فَاعِلٍ:رير:#:\nالمقبل:الْمِقْبَلِ:ال: التعريف:مقبل:اسم آلة:مِفْعَلِ:قبل:#:\nبفندق: لا توجد نتائج لتحليل هذه الكلمة\nقصر:قَصِّرْ:#:قصر:فعل أمر:فَعِّلْ:قصر:#:\nالبستان:الْبَسَّتَانِ:ال: التعريف:بستان:مصدر مرة:فَعَّتَانِ:بسس:ة:تاء التأنيث+ا علامة الإعراب:\nالمؤتمر:الْمُؤْتَمَرِ:ال: التعريف:مؤتمر:اسم زمان أو مكان:مُفْتَعَلِ:ءمر:#:\nالدولى: لا توجد نتائج لتحليل هذه الكلمة\nلفرص:لَفَرِّصْ:لَ: حرف الابتداء:فرص:فعل أمر:فَعِّلْ:فرص:#:\nالاستثمار:الْاسْتِثْمَارِ:ال: التعريف:استثمار:مصدر أصلي:اسْتِفْعَا

In [8]:
len(content)

100

In [190]:
split_list = []
for item in content : 
    tmp = item.splitlines()
    split_list.append(tmp)

In [191]:
work_list = []
for k in split_list :
    l = [item for item in k if 'لا توجد نتائج لتحليل هذه الكلمة' not in item]
    tmp_l = [item.replace("#",'') for item in l]
    work_list.append(tmp_l)

In [192]:
final_list = []
for k in work_list :
    tst = [item.split(':') for item in k]
    final_list.append(tst)

In [566]:
# identifie le prefixe, racine et le suffixe et les placent dans un dictionnaire
def identify(word_l):
    if len(word_l) < 4 :
        return None
    dictt = {}
    dictt['word'] = word_l[0]
    # le cas s'il existe un préfixe
    if word_l[2] != '' and word_l[2] != ' ' :   
        if word_l[4] not in word_l[0]: 
            if word_l[5] in word_l[0] and word_l[5] != '': 
                dictt['prefixe'] = word_l[2]
                dictt['root'] = word_l[8]
                dictt['suffixe'] = word_l[9]
            elif word_l[3] in word_l[0] and word_l[3] != '':
                dictt['prefixe'] = word_l[2]
                dictt['root'] = word_l[3]
                dictt['suffixe'] = ''
        else :
            dictt['prefixe'] = word_l[2]
            dictt['root'] = word_l[7]
            dictt['suffixe'] = word_l[8]
    # s'il n'existe pas un préfixe
    else : 
        if word_l[2] == '' : 
            dictt['prefixe'] = word_l[2]
            dictt['root'] = word_l[6]
            dictt['suffixe'] = word_l[7]
        elif  word_l[2] == ' ' :
            dictt['prefixe'] = ''
            dictt['root'] = word_l[3]
            dictt['suffixe'] = ''    
    return dictt

In [567]:
# filtre la liste de mots en liste de dictionnaires
def word_to_dict_list(wordlist):
    dictlist = []
    for k in wordlist : 
        dictlist.append(identify(k))
    return dictlist

In [568]:
final = []
for k in final_list: 
    for j in k :
        s = identify(j)
        if s == None :
            continue
        final.append(identify(j))

In [569]:
print(final[100])

{'word': 'ستقام', 'prefixe': 'سَ', 'root': 'قوم', 'suffixe': ''}


In [570]:
def dic_to_list(listt):
    L = []
    for k in listt : 
        tmp = []
        #print(k)
        if len(k) == 4 : 
            tmp.append(k['word'])
            tmp.append(k['prefixe'])
            tmp.append(k['root'])
            tmp.append(k['suffixe'])
            L.append(tmp)
    return L
data = dic_to_list(final)

In [571]:
final_l = dic_to_list(final)
final_l[0]

['مسقط', '', 'سقط', '']

In [572]:
# experimenting with our first neural network to predict the root of a word.

data_1 = []

for word in final_l:
    tmp = []
    word[0] = word[0].replace('+', ' ')
    word[2] = word[2].replace('+', ' ')
    tmp.append(word[0])
    tmp.append(word[2])
    data_1.append(tmp)
    #print(tmp)
#data_1

In [573]:
# we extract our dictionnary from the our dataset 
def extract_dict(listt) :
    dictt = []
    for word in listt :
        for item in word : 
            tmp = set(item)
            for k in tmp : 
                if k not in dictt : 
                    dictt.append(k)
    return dictt       
dic = extract_dict(data_1)

In [574]:
dic

['ق',
 'م',
 'س',
 'ط',
 'ي',
 'ا',
 'ب',
 'ع',
 'ر',
 'ل',
 'ش',
 'و',
 'ن',
 'ه',
 'ف',
 'ص',
 'ت',
 'ؤ',
 'ء',
 'ث',
 'ج',
 'ة',
 'ظ',
 'د',
 'ز',
 'ك',
 'ح',
 'ى',
 'خ',
 'غ',
 'ض',
 'أ',
 'ئ',
 'ذ',
 'آ',
 'إ']

In [575]:
root_data = []
for word in data : 
    tmp =[]
    tmp.append(word[0])
    tmp.append(word[2])
    root_data.append(tmp)
#root_data

In [21]:
class encode :
    
    def __init__(self, data):
        self.data = data
        
    def code_sequence(self):
        L = []
        for word in self.data : 
            tmp_string = code_word(word)
            L.append(tmp_string)
        final_string = '#'.join(map(str,L))
        return final_string 
    
    def code_normal_text(self):
        L = []
        for word in self.data : 
            L.append(word[1])
        final_string = ' '.join(map(str,L))
        return final_string
        
final_str = encode(root_data).code_normal_text()      

In [611]:
words = []
roots = []

for instance in data_root : 
    words.append(instance[0])
    roots.append(instance[1])


In [622]:
max([len(item) for item in words])

12

In [583]:
data_root = []
for item in root_data : 
    tmp = []
    if len(item[1]) <= 3 and len(item[1]) != 0:
        tmp.append(item[0])
        tmp.append('$'+item[1])
        data_root.append(tmp)


In [657]:
# implementation of a paper : 

import tensorflow as tf
import numpy as np
import torch.nn as nn
import torch 
import string
import torch.optim as optim
import random


class model(nn.Module): 
    
    def __init__(self, data ,words, roots, batch_size ,embedding_size, hidden_size, num_layers, teacher_forcing_ratio):
        super().__init__()
        
        '''
        our input data are arabic words with their roots, with the hypothesis that each word has it's own root.
        
        (our dataset for this is named root_data)
        '''
        self.sow = '$' # the start of root character 
        self.eow = '£' # the end of root character
        self.batch_size = batch_size
        self.data = data
        self.words = words 
        self.roots = roots
        self.embedding_size = embedding_size
        self.vocab = self.extract_dict()
        self.hidden_size = len(self.vocab)
        self.char_index_dic = self.char_to_index()
        print(self.char_index_dic)
        self.num_layers = num_layers
        self.embedding = nn.Embedding(num_embeddings = len(self.vocab), embedding_dim = self.embedding_size, padding_idx = self.char_index_dic['%']) 
        self.bigru = nn.GRU(input_size=self.embedding_size, hidden_size=self.hidden_size, num_layers=self.num_layers, bidirectional=True, batch_first=True)
        self.gru = nn.GRU(input_size= self.embedding_size ,hidden_size = self.hidden_size, num_layers = self.num_layers, batch_first = True)
        self.criterion = nn.CrossEntropyLoss()
        self.teacher_forcing_ratio = teacher_forcing_ratio
        self.Linear = nn.Linear(self.hidden_size,len(self.vocab))
        self.optimizer = optim.Adam([*self.bigru.parameters(), *self.gru.parameters()], lr = 0.1, momentum=0.9)

    
    def extract_dict(self) :
        '''
        this function extracts all the unique characters from the given dataset
        '''
        dictt = []
        for word in self.data :
            for item in word : 
                tmp = set(item)
                for k in tmp : 
                    if k not in dictt : 
                        dictt.append(k)
        dictt.append('%')
        return dictt 
    
    
    # Let's now construct our model : 
    
    # we should think about character embeddings in order to create an embeded matrix for each word
    
    def char_to_index(self):
        '''
        this function creates unique indexes of each character
        '''
        char_to_idx_map = {char: idx for idx, char in enumerate(self.vocab)}
        return char_to_idx_map  
    
    def data_batches(self): 
        
        size = self.batch_size
        
        batches = [self.data[i:i + size] for i in range(0, len(self.data), size)]
        '''word_batches = [self.words[i:i + size] for i in range(0, len(self.words), size)]
        root_batches = [self.roots[i:i + size] for i in range(0, len(self.roots), size)]'''

        
        return batches
    
    def batch_separator(self, batch):
        
        words, roots =[], []
        for inst in batch : 
            words.append(inst[0])
            roots.append(inst[1])
        return words , roots
            
        
    
    def batch_2_seq(self, batch, mode):
        '''
        this function returns a tensor of sequences of the unique indexes for the given batch 
        (sequence is tensor that can be changed using a .tolist() )
        '''
        if mode == 'encode':
            max_len = max([len(item) for item in self.words])
        else :
            max_len = max([len(item) for item in self.roots])
        
        tmp = []
        for ins in batch :
            #print(ins)
            item = [self.char_index_dic[char] for char in ins]
            while(len(item) < max_len) : 
                item.append(self.char_index_dic['%'])
            tmp.append(item)

        batch_seq_tensor = torch.tensor(tmp)        
        return batch_seq_tensor # word sequence
    
    def word_to_seq(self, word):
        '''
        this function returns a sequence of the unique indexes for the given word 
        (sequence is tensor that can be changed using a .tolist() )
        '''
        word_char_idx_tensor =[self.char_index_dic[char] for char in word]    
        return word_char_idx_tensor # word sequence
    
    
    def encode(self, batch):
        
            
        # we create embedding of the sequence : 
        batch_seq = self.batch_2_seq(batch, 'encode')
        print(batch_seq.size())
        embedded_batch = self.embedding(batch_seq)
        
        print(embedded_batch.size())
        
        outputs, hidden = self.bigru(embedded_batch) # we pass the emebedded vector through the bi-GRU 
        
        # can also be outputs.
        
        print(hidden.size())
        
        encoder_output = torch.mean(hidden , dim=0)  # Average the hidden vectors across all time steps. Shape: (hidden_size*2,) if bidirectional, else (hidden_size,)
        
        final_output = torch.unsqueeze(encoder_output, 0)
        return final_output

    
    def test(self):
        
        self.optimizer.zero_grad()
        
        batches = self.data_batches()
        for epoch in epochs :
            all_pos = []
            for batch in batches :
                word_batches, root_batches = self.batch_separator(batch)
                tst = self.encode(word_batches)
                print(tst.size())
                res_char , hidden , positions  = self.decode(tst, root_batches)
                all_pos.append(positions)
    
    
    def decode(self, encoding, batch):

        
        '''
        encoding : output of the encoder network. 
        
        '''
        root_batch = self.batch_2_seq(batch, 'decode')
        
        embedded_char = self.embedding(root_batch)   # starts with self.sow
                
        input_size = embedded_char.size(1)

        hidden_size = encoding.size(1)
            
        print(embedded_char.size(), encoding.size())
            
        dec_out , dec_hidden = self.gru(embedded_char ,encoding)
        
        print(dec_out.size())
        
        a = self.Linear(dec_out)
        m = nn.Softmax(dim = 2)
        
        output = m(a)
        
        print(output.size())
        
        
        tops = [item.argmax(1)[0].item() for item in output]
        
        print(tops)
        
        #top1 = output.argmax(1)[0].item()
        
        key_list = list(self.char_index_dic.keys())
        val_list = list(self.char_index_dic.values())

        positions = [val_list.index(item) for item in tops]
        
        result_chars = [key_list[pos] for pos in positions]
        print(result_chars)
        
        #print(top1, position)

        return result_chars, dec_hidden , positions
        
        
    def train_batch(self, batch):
                
        self.optimizer.zero_grad()
        for instance in batch: 
            '''
            word = instance[0] : the word to extract root from
            target_word = instance[1] : the root of the word
            '''
            word = instance[0]
            target_root = instance[1]
            
            res1 = self.encode_word(word)
            predicted_root = []
            original_root_seq = self.word_to_idx_seq(target_root)
            predicted_root_seq = []
            res_char = target_root[0]
            
            i = 0
            for char in target_root : 
                if len(predicted_root) == len(target_root):
                    break
                
                
                if random.random() < self.teacher_forcing_ratio and i!=0 : 
                    res_char = char
                    i+= 1
                
                res_char , hidd , idx = self.decode_word(res1, res_char)
                predicted_root_seq.append(idx)
                test = hidd
                predicted_root.append(res_char)

            predicted_root = ''.join(predicted_root)

            predicted_root_seq = torch.tensor(predicted_root_seq)

            org = self.embedding(original_root_seq)
            pred = self.embedding(predicted_root_seq)
            
            
            loss = self.criterion(org, pred)
            
            loss.backward()

            print('target root :', target_root)
            print('predicted root : ', predicted_root)

            print('the loss : ', loss.item())  
        self.optimizer.step()
        
        return
    
    
    def train(self):
        
        batches = self.data_batches()
        iter = 0
        for batch in batches : 
            
            self.train_batch(batch)
            print('*'*100)
            print(iter)
            iter+=1
            
            
        #print(em1)
        
        

mod = model(data_root,words , roots,  100, 50, 100, 1, 0.5)
mod.test()

{'ق': 0, 'م': 1, 'س': 2, 'ط': 3, '$': 4, 'ي': 5, 'ا': 6, 'ب': 7, 'ع': 8, 'ر': 9, 'ل': 10, 'ش': 11, 'و': 12, 'ن': 13, 'ه': 14, 'ف': 15, 'ص': 16, 'ت': 17, 'ؤ': 18, 'ء': 19, 'ث': 20, 'ج': 21, 'ة': 22, 'ظ': 23, 'د': 24, 'ز': 25, 'ك': 26, 'ح': 27, 'ى': 28, 'خ': 29, 'غ': 30, 'ض': 31, 'أ': 32, 'ئ': 33, 'ذ': 34, 'آ': 35, 'إ': 36, '%': 37}


TypeError: Adam.__init__() got an unexpected keyword argument 'momentum'

tensor([[1, 2, 3],
        [4, 8, 7]])

In [599]:
# implementation of a paper : 

import tensorflow as tf
import numpy as np
import torch.nn as nn
import torch 
import string
import torch.optim as optim
import random


class model(nn.Module): 
    
    def __init__(self, data, batch_size ,embedding_size, hidden_size, num_layers, teacher_forcing_ratio):
        super().__init__()
        
        '''
        our input data are arabic words with their roots, with the hypothesis that each word has it's own root.
        
        (our dataset for this is named root_data)
        '''
        self.sow = '$' # the start of root character 
        self.eow = '£' # the end of root character
        self.batch_size = batch_size
        self.data = data
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.vocab = self.extract_dict()
        self.char_index_dic = self.char_to_index()
        print(self.char_index_dic)
        self.num_layers = num_layers
        self.embedding = nn.Embedding(num_embeddings = len(self.vocab), embedding_dim = self.embedding_size) 
        self.bigru = nn.GRU(input_size=self.embedding_size, hidden_size=self.hidden_size, num_layers=self.num_layers, bidirectional=True)
        self.gru = nn.GRU(input_size= self.embedding_size ,hidden_size = self.hidden_size, num_layers = self.num_layers)
        self.criterion = nn.CrossEntropyLoss()
        self.teacher_forcing_ratio = teacher_forcing_ratio
        self.Linear = nn.Linear(self.hidden_size,len(self.vocab))
        self.optimizer = optim.Adam([*self.bigru.parameters(), *self.gru.parameters(), *self.Linear.parameters()], lr = 0.1)

    
    def extract_dict(self) :
        '''
        this function extracts all the unique characters from the given dataset
        '''
        dictt = []
        for word in self.data :
            for item in word : 
                tmp = set(item)
                for k in tmp : 
                    if k not in dictt : 
                        dictt.append(k)
        return dictt 
    
    
    # Let's now construct our model : 
    
    # we should think about character embeddings in order to create an embeded matrix for each word
    
    def char_to_index(self):
        '''
        this function creates unique indexes of each character
        '''
        char_to_idx_map = {char: idx for idx, char in enumerate(self.vocab)}
        return char_to_idx_map  
    
    def data_batches(self): 
        size = self.batch_size
        
        batches = [self.data[i:i + size] for i in range(0, len(self.data), size)]
        
        return batches
        
    
    def word_to_idx_seq(self, word):
        '''
        this function returns a sequence of the unique indexes for the given word 
        (sequence is tensor that can be changed using a .tolist() )
        '''
        word_char_idx_tensor = torch.tensor([self.char_index_dic[char] for char in word])        
        return word_char_idx_tensor # word sequence
    
    def word_to_seq(self, word):
        '''
        this function returns a sequence of the unique indexes for the given word 
        (sequence is tensor that can be changed using a .tolist() )
        '''
        word_char_idx_tensor =[self.char_index_dic[char] for char in word]    
        return word_char_idx_tensor # word sequence
    
    
    def encode_word(self, word):
        
            
        # we create embedding of the sequence : 
        word_seq = self.word_to_idx_seq(word)
        embedded_vec = self.embedding(word_seq)
        
        outputs, hidden = self.bigru(embedded_vec) # we pass the emebedded vector through the bi-GRU 

        
        '''
        kaynin two cases : 
        
             case1 :  we work on the outputs  ==> chosen
             case2 :  we work on the final hidden state ==> discarted 
            
        '''
        
        
        '''test_hidden = torch.flatten(hidden)
        final_hidden = torch.unsqueeze(test_hidden, 0)'''
        
        
        # can also be outputs.
        
        encoder_output = torch.mean(hidden , dim=0)  # Average the hidden vectors across all time steps. Shape: (hidden_size*2,) if bidirectional, else (hidden_size,)
        
        final_output = torch.unsqueeze(encoder_output, 0)
               
        
        return final_output
    
    
    def decode_word(self, encoding, character):

        
        '''
        encoding : output of the encoder network. 
        
        '''
        
        seq = self.word_to_idx_seq(self.sow)
        embedded_sow= self.embedding(self.word_to_idx_seq(character))   # starts with self.sow

        input_size = embedded_sow.size(1)

        hidden_size = encoding.size(1)
                 
        dec_out , dec_hidden = self.gru(embedded_sow,encoding)
                
        a = self.Linear(dec_out)
        m = nn.Softmax(dim = 1)
        
        output = m(a)
        
        top1 = output.argmax(1)[0].item()
        
        key_list = list(self.char_index_dic.keys())
        val_list = list(self.char_index_dic.values())

        position = val_list.index(top1)
        
        result_char = key_list[position]

        return result_char, dec_hidden , top1
        
        
    def train_batch(self, batch):
                
        
        for instance in batch: 
            '''
            word = instance[0] : the word to extract root from
            target_word = instance[1] : the root of the word
            '''
            word = instance[0]
            target_root = instance[1]
            
            res1 = self.encode_word(word)
            predicted_root = []
            original_root_seq = self.word_to_idx_seq(target_root)
            predicted_root_seq = []
            res_char = target_root[0]
            for char in target_root : 
                if len(predicted_root) == len(target_root):
                    break
                
                
                if random.random() < self.teacher_forcing_ratio : 
                    res_char = char
                
                res_char , hidd , idx = self.decode_word(res1, res_char)
                predicted_root_seq.append(idx)
                test = hidd
                predicted_root.append(res_char)

            predicted_root = ''.join(predicted_root)

            predicted_root_seq = torch.tensor(predicted_root_seq)

            org = self.embedding(original_root_seq)
            pred = self.embedding(predicted_root_seq)
            
            self.optimizer.zero_grad()
            loss = self.criterion(org, pred)
            
            loss.backward()
            self.optimizer.step()

            print('target root :', target_root)
            print('predicted root : ', predicted_root)

            print('the loss : ', loss.item())        
        
        return None
    
    
    def train(self):
        
        batches = self.data_batches()
        
        for batch in batches : 
            
            self.train_batch(batch)

        #print(em1)
        
        

mod = model(data_root, 100, 50, 100, 1, 0.9)
mod.train()

{'ق': 0, 'م': 1, 'س': 2, 'ط': 3, '$': 4, 'ي': 5, 'ا': 6, 'ب': 7, 'ع': 8, 'ر': 9, 'ل': 10, 'ش': 11, 'و': 12, 'ن': 13, 'ه': 14, 'ف': 15, 'ص': 16, 'ت': 17, 'ؤ': 18, 'ء': 19, 'ث': 20, 'ج': 21, 'ة': 22, 'ظ': 23, 'د': 24, 'ز': 25, 'ك': 26, 'ح': 27, 'ى': 28, 'خ': 29, 'غ': 30, 'ض': 31, 'أ': 32, 'ئ': 33, 'ذ': 34, 'آ': 35, 'إ': 36}
target root : $سقط
predicted root :  خسسص
the loss :  -12.39161491394043
target root : $قمي
predicted root :  خهطو
the loss :  0.7388191223144531
target root : $ربع
predicted root :  خآحب
the loss :  -15.721403121948242
target root : $عشر
predicted root :  كضءك
the loss :  -19.307065963745117
target root : $من
predicted root :  خصؤ
the loss :  7.9825310707092285
target root : $شهر
predicted root :  خبخم
the loss :  1.8922204971313477
target root : $رير
predicted root :  خحهح
the loss :  -21.36949920654297
target root : $قبل
predicted root :  خهذخ
the loss :  -9.17136287689209
target root : $قصر
predicted root :  إهخم
the loss :  2.0795624256134033
target root : $بسس
p

target root : $عدد
predicted root :  خبةة
the loss :  -24.82905387878418
target root : $من
predicted root :  خصؤ
the loss :  7.9825310707092285
target root : $رءس
predicted root :  بكحض
the loss :  -27.556884765625
target root : $شرك
predicted root :  ؤبزآ
the loss :  4.818280220031738
target root : $صلل
predicted root :  خخزآ
the loss :  -5.430098533630371
target root : $خدم
predicted root :  ببرط
the loss :  -7.157011032104492
target root : $وعم
predicted root :  كمبص
the loss :  3.3336215019226074
target root : $عدد
predicted root :  خبةة
the loss :  -24.82905387878418
target root : $من
predicted root :  خصؤ
the loss :  7.9825310707092285
target root : $شرك
predicted root :  ببكم
the loss :  0.2761564254760742
target root : $وصل
predicted root :  ؤضءز
the loss :  -9.810297012329102
target root : $في
predicted root :  خمذ
the loss :  4.263490676879883
target root : $بدي
predicted root :  بذرآ
the loss :  -7.753918647766113
target root : $لقي
predicted root :  بذهه
the loss :  -8.3693

target root : $عمل
predicted root :  خضطص
the loss :  -10.549362182617188
target root : $خلل
predicted root :  خبمم
the loss :  11.964178085327148
target root : $فتر
predicted root :  بحضك
the loss :  -28.12701988220215
target root : $قدم
predicted root :  بهرك
the loss :  -14.674612045288086
target root : $حقق
predicted root :  ؤممم
the loss :  29.853450775146484
target root : $كمل
predicted root :  خمطص
the loss :  5.656820297241211
target root : $بين
predicted root :  خذصؤ
the loss :  10.49303150177002
target root : $صنع
predicted root :  بخب$
the loss :  -2.488602876663208
target root : $سمك
predicted root :  بذطم
the loss :  1.3256627321243286
target root : $في
predicted root :  خمص
the loss :  10.242226600646973
target root : $بلد
predicted root :  ذذصص
the loss :  2.968376636505127
target root : $حقق
predicted root :  بمطه
the loss :  -2.037139415740967
target root : $سوو
predicted root :  خسضض
the loss :  -45.30415725708008
target root : $كبر
predicted root :  خمحم
the loss :  

target root : $علق
predicted root :  ب$جه
the loss :  -12.236661911010742
target root : $طور
predicted root :  خصخم
the loss :  1.5479602813720703
target root : $عمل
predicted root :  خضطص
the loss :  -10.549362182617188
target root : $في
predicted root :  خمص
the loss :  10.242226600646973
target root : $قطع
predicted root :  بهقب
the loss :  3.711064338684082
target root : $سمك
predicted root :  بصطم
the loss :  8.086739540100098
target root : $بين
predicted root :  خذصؤ
the loss :  10.49303150177002
target root : $رجل
predicted root :  خآخخ
the loss :  -14.882537841796875
target root : $ءسس
predicted root :  بةسس
the loss :  -34.89342498779297
target root : $قطع
predicted root :  بهصب
the loss :  -3.324469566345215
target root : $خصص
predicted root :  ببخخ
the loss :  -2.410754680633545
target root : $في
predicted root :  خمص
the loss :  10.242226600646973
target root : $وكل
predicted root :  خضمذ
the loss :  -13.007180213928223
target root : $من
predicted root :  خصؤ
the loss :  7.

target root : $علي
predicted root :  خبمه
the loss :  -2.5312647819519043
target root : $بذذ
predicted root :  بذكك
the loss :  -1.4160945415496826
target root : $قرب
predicted root :  خهآخ
the loss :  -13.067405700683594
target root : $عطف
predicted root :  ببصم
the loss :  6.206565856933594
target root : $علي
predicted root :  خضمه
the loss :  -12.87913703918457
target root : $رفق
predicted root :  خآأك
the loss :  -15.273580551147461
target root : $بهم
predicted root :  خحخط
the loss :  -9.403340339660645
target root : $يدي
predicted root :  خةصة
the loss :  -24.345563888549805
target root : $مسح
predicted root :  خطسم
the loss :  -8.736876487731934
target root : $رءس
predicted root :  خبةء
the loss :  -16.788475036621094
target root : $سوك
predicted root :  ؤصضم
the loss :  4.9789934158325195
target root : $بذل
predicted root :  خذمئ
the loss :  -6.599379539489746
target root : $سير
predicted root :  خحمط
the loss :  -3.117487907409668
target root : $رود
predicted root :  خكمة
the 

...

target root : $مثل
predicted root :  خهخخ
the loss :  -10.14373779296875
target root : $وحد
predicted root :  بضمر
the loss :  -19.557039260864258
target root : $دول
predicted root :  خضضج
the loss :  -40.64908981323242
target root : $سير
predicted root :  ؤءهز
the loss :  -3.340564727783203
target root : $رءس
predicted root :  خآةس
the loss :  -29.552207946777344
target root : $جنن
predicted root :  ؤ$ثث
the loss :  7.976276397705078
target root : $حكم
predicted root :  بممط
the loss :  8.999320983886719
target root : $في
predicted root :  خمص
the loss :  10.242226600646973
target root : $وعم
predicted root :  خببص
the loss :  -2.516975164413452
target root : $دول
predicted root :  خضضج
the loss :  -40.64908981323242
target root : $عيم
predicted root :  بذهط
the loss :  -6.783407688140869
target root : $على
predicted root :  خآم$
the loss :  5.766806125640869
target root : $جوح
predicted root :  بحضم
the loss :  -22.47740936279297
target root : $في
predicted root :  خمص
the loss :  10

target root : $ما
predicted root :  بهب
the loss :  -3.1797988414764404
target root : $جوب
predicted root :  خ$هذ
the loss :  -5.935698986053467
target root : $وفر
predicted root :  خضمح
the loss :  -18.17336654663086
target root : $دعم
predicted root :  بصبط
the loss :  -1.5787780284881592
target root : $كبر
predicted root :  خبحك
the loss :  -10.560286521911621
target root : $مثل
predicted root :  خبخز
the loss :  -8.412015914916992
target root : $هذه
predicted root :  ؤضمض
the loss :  -14.321867942810059
target root : $بطل
predicted root :  خبصئ
the loss :  -24.940589904785156
target root : $ضخم
predicted root :  بحبط
the loss :  -10.276196479797363
target root : $هجم
predicted root :  خخمط
the loss :  5.040186882019043
target root : $جدد
predicted root :  خبةة
the loss :  -23.424009323120117
target root : $على
predicted root :  خآم$
the loss :  5.766806125640869
target root : $سلم
predicted root :  بيمط
the loss :  9.538886070251465
target root : $في
predicted root :  خبص
the loss 

target root : $أن
predicted root :  خكم
the loss :  5.146701335906982
target root : $كون
predicted root :  خمضؤ
the loss :  -3.835907459259033
target root : $حلل
predicted root :  بمطم
the loss :  7.402464866638184
target root : $نحن
predicted root :  خؤمؤ
the loss :  21.794679641723633
target root : $سلم
predicted root :  ضصصط
the loss :  -7.559370040893555
target root : $على
predicted root :  خآم$
the loss :  5.766806125640869
target root : $نقض
predicted root :  ذؤهأ
the loss :  3.669966459274292
target root : $من
predicted root :  خصؤ
the loss :  7.9825310707092285
target root : $مرر
predicted root :  خصكك
the loss :  -6.081638813018799
target root : $ودي
predicted root :  خبنة
the loss :  -15.32356071472168
target root : $ودي
predicted root :  خضةة
the loss :  -39.13332748413086
target root : $ءمر
predicted root :  خةطك
the loss :  -13.901688575744629
target root : $أن
predicted root :  خبؤ
the loss :  9.970629692077637
target root : $وحد
predicted root :  خضمص
the loss :  -14.565

target root : $جيء
predicted root :  ب$هح
the loss :  -15.395269393920898
target root : $بوه
predicted root :  خذضأ
the loss :  -24.436260223388672
target root : $قرء
predicted root :  ؤهكة
the loss :  -2.3843207359313965
target root : $جيء
predicted root :  ب$ةة
the loss :  -21.99624252319336
target root : $بوه
predicted root :  خذضض
the loss :  -36.48462677001953
target root : $سنن
predicted root :  خصؤؤ
the loss :  20.882099151611328
target root : $لكن
predicted root :  كصمؤ
the loss :  18.333541870117188
target root : $ذلل
predicted root :  خمصص
the loss :  10.05369758605957
target root : $حوج
predicted root :  بمض$
the loss :  -11.510536193847656
target root : $من
predicted root :  خصؤ
the loss :  7.9825310707092285
target root : $وكل
predicted root :  خبمذ
the loss :  5.44406795501709
target root : $عصر
predicted root :  هآخه
the loss :  0.6169586181640625
target root : $من
predicted root :  خكؤ
the loss :  4.771125793457031
target root : $وكل
predicted root :  خضمذ
the loss :  -

KeyboardInterrupt: 

In [401]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
output.backward()
print(output)

tensor(2.2498, grad_fn=<DivBackward1>)


In [530]:
'''l = 0
for item in root_data:
    if len(item[1]) > 3:
        print(item[1])'''

'l = 0\nfor item in root_data:\n    if len(item[1]) > 3:\n        print(item[1])'

In [24]:
'''def decode_word(self, word):
        
        hidden_size = 32  # Number of hidden units in the GRU
        num_layers = 1  # Number of layers in the GRU
        dropout = 0.2  # Dropout rate
        
        gru = nn.GRU(input_size=self.embedding_size, hidden_size=hidden_size, batch_first=True)
        
        e = self.encode_word(word)
        # Initialize the hidden state h0
        
        h0 = torch.zeros(1, hidden_size)

        
        # Concatenate the encoder representation and start-of-word character
        print(e.size())
        print(self.hsowi.unsqueeze(1).size())
        
        x = torch.cat([self.hsowi.unsqueeze(0).float(), e.unsqueeze(0)], dim=1)

        print(x.size())
        print(h0.size())
        
        # Apply the GRU layer
        out, h = gru(x)
        
        # Apply the output layer
        out_l = nn.Linear(hidden_size, len(self.vocab))
        
        out = out_l(out)
        
        softmax = nn.Softmax(dim=2)
        
        # Apply the softmax activation function
        out = softmax(out)
        
        return out
    '''

'def decode_word(self, word):\n        \n        hidden_size = 32  # Number of hidden units in the GRU\n        num_layers = 1  # Number of layers in the GRU\n        dropout = 0.2  # Dropout rate\n        \n        gru = nn.GRU(input_size=self.embedding_size, hidden_size=hidden_size, batch_first=True)\n        \n        e = self.encode_word(word)\n        # Initialize the hidden state h0\n        \n        h0 = torch.zeros(1, hidden_size)\n\n        \n        # Concatenate the encoder representation and start-of-word character\n        print(e.size())\n        print(self.hsowi.unsqueeze(1).size())\n        \n        x = torch.cat([self.hsowi.unsqueeze(0).float(), e.unsqueeze(0)], dim=1)\n\n        print(x.size())\n        print(h0.size())\n        \n        # Apply the GRU layer\n        out, h = gru(x)\n        \n        # Apply the output layer\n        out_l = nn.Linear(hidden_size, len(self.vocab))\n        \n        out = out_l(out)\n        \n        softmax = nn.Softmax(dim=2)\

In [215]:
'''import torch
import torch.nn as nn


class model:
    def __init__(self, data, embedding_size):
        self.sow = '$'  # the start of word character
        self.eow = '£'  # the end of word character
        self.data = data
        self.embedding_size = embedding_size
        self.vocab = self.extract_dict()
        self.char_index_dic = self.char_to_index()
        self.embedding = nn.Embedding(num_embeddings=len(self.vocab), embedding_dim=self.embedding_size)

    def extract_dict(self):
        dictt = []
        for word in self.data:
            for item in word:
                tmp = set(item)
                for k in tmp:
                    if k not in dictt:
                        dictt.append(k)
        dictt.append(self.sow)
        dictt.append(self.eow)
        return dictt

    def char_to_index(self):
        char_to_idx_map = {char: idx for idx, char in enumerate(self.vocab)}
        #print(char_to_idx_map)
        return char_to_idx_map

    def word_to_idx_seq(self, word):
        word_char_idx_tensor = torch.tensor([self.char_index_dic[char] for char in word])
        return word_char_idx_tensor

    def encode_word(self, word):
        
        hidden_size = 32
        bidirectional = True
        num_layers = 1
        
        
        gru = nn.GRU(input_size=self.embedding_size, hidden_size=hidden_size, num_layers=num_layers,
                     bidirectional=bidirectional, batch_first=True)

        
        
        word_seq = self.word_to_idx_seq(word)
        embedded_vec = self.embedding(word_seq)
        outputs, hidden = gru(embedded_vec)

        h1 = torch.reshape(hidden[-1], (1, hidden_size))
        h2 = torch.reshape(hidden[-2], (1, hidden_size))
        hidden = torch.cat((h2, h1), 1)

        encoder_output = torch.mean(hidden, dim=0)

        return encoder_output

    def decode_word(self, encoding):
        input_size = encoding.shape[-1] + len(self.vocab)
        hidden_size = 64
        num_layers = 1

        embedded_sow = self.embedding(torch.tensor(self.char_index_dic[self.sow]))
        decoder_input = torch.cat((encoding, embedded_sow), dim=1)

        gru = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)

        outputs, hidden = gru(decoder_input.unsqueeze(0))
        decoder_output = torch.mean(hidden, dim=0)

        decoded = []
        for i in range(self.max_word_length):
            output = self.output_layer(decoder_output)
            _, argmax = torch.max(output, dim=1)
            decoded.append(argmax.item())
            decoder_input = self.embedding(argmax)
            decoder_input = torch.cat((decoder_input, encoding), dim=1)
            decoder_output, hidden = gru(decoder_input.unsqueeze(0), hidden)
            decoder_output = torch.mean(hidden, dim=0)

            if argmax == self.eow_index:
                break

        decoded_word = ' '.join([self.index_char_dic[i] for i in decoded])
        return decoded_word


    def train(self):
        #for instance in self.data:
        word = self.data[0]
        encoding = self.encode_word(word)
        print(f'Encoding for {word}: {encoding}')

        decoding = self.decode_word(encoding)
        print(f'Decoding for {word}: {decoding}')
        break

mod = model(root_data, 10)
mod.train()'''

"import torch\nimport torch.nn as nn\n\n\nclass model:\n    def __init__(self, data, embedding_size):\n        self.sow = '$'  # the start of word character\n        self.eow = '£'  # the end of word character\n        self.data = data\n        self.embedding_size = embedding_size\n        self.vocab = self.extract_dict()\n        self.char_index_dic = self.char_to_index()\n        self.embedding = nn.Embedding(num_embeddings=len(self.vocab), embedding_dim=self.embedding_size)\n\n    def extract_dict(self):\n        dictt = []\n        for word in self.data:\n            for item in word:\n                tmp = set(item)\n                for k in tmp:\n                    if k not in dictt:\n                        dictt.append(k)\n        dictt.append(self.sow)\n        dictt.append(self.eow)\n        return dictt\n\n    def char_to_index(self):\n        char_to_idx_map = {char: idx for idx, char in enumerate(self.vocab)}\n        #print(char_to_idx_map)\n        return char_to_idx_ma

In [ ]:
import tensorflow as tf

# create an embedding layer with 50 input dimensions and 100 output dimensions
embedding_layer = tf.keras.layers.Embedding(input_dim=43, output_dim=100)
sequence = [5 , 10 , 10, 20 , 7, 1]
# get the embedding vectors for the sequence of character indices
embedding_vectors = embedding_layer(np.array(sequence))
embedding_vectors

In [ ]:
hidden_size = 32  # Number of hidden units in the GRU
bidirectional = True  # Whether to use a bidirectional GRU or not
num_layers = 1  # Number of layers in the GRU
dropout = 0.2  # Dropout rate
gru = nn.GRU(input_size=16, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True)


In [ ]:
import torch
import torch.nn as nn
import string

In [ ]:
char_to_idx_map = {char: idx for idx, char in enumerate(dic)}
print(f"Character to Index Mapping:\n{char_to_idx_map}\n")

In [ ]:
ohe_characters = torch.eye(n=len(dic))  # using the eye method for identity matrix
print(f"One hot encoded characters:\n{ohe_characters}\n")

In [ ]:
ohe_characters.shape

In [ ]:
ohe_repr_a = ohe_characters[char_to_idx_map['ح']]
print(f"One hot encoded representation of 1:\n{ohe_repr_a}\n")

In [ ]:
def encode_word(word, char_to_idx_map):
    
    word_char_idx_tensor = torch.tensor([char_to_idx_map[char] for char in word])
    word_ohe_repr = ohe_characters[word_char_idx_tensor].T
    
    return word_ohe_repr

In [ ]:
word_char_idx_tensor = torch.tensor([char_to_idx_map[char] for char in "مَرْحَبًا"])
print(word_char_idx_tensor.tolist())
word_ohe_repr = ohe_characters[word_char_idx_tensor].T
word_ohe_repr.shape

In [ ]:
convolution_layer = nn.Conv1d(in_channels=43, out_channels=128, kernel_size=3, bias=False)
conv_out = convolution_layer(word_ohe_repr.unsqueeze(0))

In [ ]:
print(f"Shape of convolution output: {conv_out.shape}")
print(f"Convolution Output:\n{conv_out}\n")

# adding an activation layer so as to add non linearity to the output
activation_layer = nn.Tanh()
activation_out = activation_layer(conv_out)
activation_out

In [ ]:
print(f"Shape of activation output: {activation_out.shape}")
print(f"Activation Output:\n{activation_out}")

In [ ]:
# Max Pooling Layer
max_pooling_layer = nn.MaxPool1d(kernel_size=conv_out.shape[2])

# finally here is our word embedding of size -> 256
word_embedding = max_pooling_layer(activation_out).squeeze()

print(f"Final word embedding shape: {word_embedding.shape}")
print(f"Final word embedding:\n{word_embedding}")

In [ ]:
def character_embedding():
    

In [559]:
st = ''.join(dic)
vowel_pattern = r'[\u064e\u064b\u0650\u064d\u064f\u064c\u064f\u0652\u0651]'
vowels = re.findall(vowel_pattern, st)
l = []
for char in st :
    if char not in vowels : 
        l.append(char)
        
l = ''.join(l)
l

'مسطقيابعرلشونهفصتؤءثجةظدزكحىخغضأئذآإ'

### Let's create a data structure for each word of the corpus

In [ ]:
tmp = work_list[2]
tmp.split(":")

In [ ]:
vowel_pattern = r'[\u064b\u064e\u064f\u0650\u0651\u0652]'
word = "مَرْحَبًا"
vowels = re.findall(vowel_pattern, word)
print(f"Vowels in '{word}': {vowels}")

In [ ]:
'''filtered_split_list = []
for item in split_list : 
    temp_item_list = []
    for subitem in item : 
        if 'لا توجد نتائج لتحليل هذه الكلمة' not in item : 
            temp_item_list.append(item)
    filtered_split_list.append(temp_item_list)
filtered_split_list[0]'''

In [ ]:
'''# Check the data type of each element in texts
texts = pd.DataFrame(data_1[:, -1])
texts = texts.iloc[:, 1:]
for i, text in enumerate(texts):
    if not isinstance(text, str):
        print(f"Element {i} is not a string: {text}")'''